In [1]:
import pdfplumber
import fitz
import camelot
import warnings , math, collections , os, re
import pickle
import numpy as np

warnings.filterwarnings("ignore", category=UserWarning) 

In [2]:
path = r"C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\mywork-repo"
#path = r"C:\Users\rando\OneDrive\Documents\mywork-repo"

#file data paths
samco_path = path + r"\files\SamcoFactSheet2024.pdf"

#dry run paths
dry_run_path = path + r"\output\DryRun.pdf"


In [3]:
#SAMCO PATHS
#samco output path
no_image_path = path + r"\output\sam\NoImgPdf.pdf"
textual_pdf_path = path + r"\output\sam\TextualPdf.pdf"
tabular_pdf_path = path + r"\output\sam\TabularPdf.pdf"

#pickle data paths samco
pickle_text = r"\output\pkl\sam\textual_data.pkl"
pickle_tab = r"\output\pkl\sam\tabular_data.pkl"
pickle_nonimg = r"\output\pkl\sam\nonimg_data.pkl"

In [4]:
pages = [3,5,7,9,11,13,15]
bbox = [(31,15,575,115),(35,120,250,765)] #for header and other for content


def get_data_clipped(input:str, output:str, pageSelect:list, bbox:list[set]):
    
    document = fitz.open(input)
    finalData = []
    
    for pgn, pages in enumerate(document):
        if pgn in pageSelect:
            page = document[pgn]
            
            title_blocks = page.get_text('dict', clip = bbox[0])['blocks'] #get title
            blocks = page.get_text('dict', clip = bbox[1])['blocks'] #get all blocks
            filtered_blocks = [block for block in title_blocks if block['type']==0] + [block for block in blocks if block['type']==0]
            sorted_blocks = sorted(filtered_blocks, key= lambda x: (x['bbox'][1], x['bbox'][0]))
            finalData.append({
                "page": pgn,
                "block": sorted_blocks
            })
            
    return finalData

In [5]:
data = get_data_clipped(samco_path, dry_run_path, pages, bbox)

In [ ]:
data[2]['block']

In [ ]:
for block in data[2]['block']:
    if 'lines' in block:
        for lines in block['lines']:
            for span in lines['spans']:
                print(span['size'])

In [ ]:
final_text_data = []
for page in data:
    page_content = []
    for blocks in page['block']:
        if 'lines' in blocks:
            for line in blocks['lines']:
                spans = line.get('spans',[])
                text = "".join(span['text'] for span in spans)
                page_content.append(text)
    
    final_text_data.append(page_content)           
    
            
final_text_data    

In [6]:
"""Recursively remove specific keys from nested dictionaries or lists.
    Args:data (dict | list): The input data (nested structure of dictionaries and lists).
    keys_to_remove (set): Keys to be removed from the dictionaries.
    Returns:dict | list: Data with specified keys removed."""
def remove_keys(data:list, keys_to_remove:set):
    if isinstance(data, list):
        # Process each element in the list
        return [remove_keys(item, keys_to_remove) for item in data]
    elif isinstance(data, dict):
        # Process each key-value pair in the dictionary
        return {key: remove_keys(value, keys_to_remove) for key, value in data.items() if key not in keys_to_remove}
    else:
        # Return data as is if it's neither a dict nor a list
        return data

In [7]:
keys_to_remove = {}
cleaned_data = remove_keys(data, keys_to_remove)

In [ ]:
cleaned_data[3]['block']

In [8]:
def clean_data_combine_spans(data:list, check_color:int,replace_size):
    
    #remove redundant text
    texts_to_remove = [':','st',";","-",'st ',' ']
    for cleaned_data in data:
        for block in cleaned_data.get("block", []):
                for line in block.get("lines", []):
                    # Filter out spans whose text matches any of the texts to remove
                    line["spans"] = [
                        span for span in line.get("spans", [])
                        if span.get("text").strip() not in texts_to_remove
                    ] 
                    
    #make size of text with same color equal to size
    for cleaned_data in data:
        for block in cleaned_data.get("block", []):
                for line in block.get("lines", []):
                    for span in line.get('spans',[]):
                        if span['color'] == check_color and span['size']> 7:
                            span['size'] = replace_size              
                        
    #combine spans with same font, flag,size,color etc
    for cleaned_data in data:
        for block in cleaned_data.get("block", []):
            for line in block.get("lines", []):
                combined_spans = []
                for span in line.get("spans", []):
                    if combined_spans and all(
                        combined_spans[-1].get(key) == span.get(key)
                        for key in ["flags", "size", "color"]
                    ):
                        # Combine text if spans are similar
                        combined_spans[-1]["text"] += " " + span["text"]
                    else:
                        combined_spans.append(span)
                line["spans"] = combined_spans
    
    return data

combined_data = clean_data_combine_spans(cleaned_data, -1, 9.0) #set font whose color is __ to size __ 
#Bad Logic here btw

In [ ]:
combined_data[3]['block'] #Page 9

In [9]:
def create_matrix_structure(data:list, title_font:float, subheader_font:float, content_max_font:float):
    # Step 1: Extract all unique coordinates
    coordinates = []
    fonts = set()

    for block in data['block']:
        for line in block['lines']:
            for span in line['spans']:
                origin = tuple(span['origin'])  # Extract (x, y) coordinates from origin
                coordinates.append(origin)
                fonts.add(span['size'])

    coordinates = sorted(set(coordinates), key=lambda c: (c[1], c[0]))  # Sort by y, then x
    fonts = sorted(fonts)

    # Step 2: Initialize the matrix
    coord_to_index = {coord: idx for idx, coord in enumerate(coordinates)}
    font_to_index = {font: idx for idx, font in enumerate(fonts)}
    matrix = np.zeros((len(coordinates), len(fonts)), dtype=object)

    # Step 3: Fill the matrix
    for block in data['block']:
        for line in block['lines']:
            for span in line['spans']:
                origin = tuple(span['origin'])  # Get (x, y) coordinates from origin
                font = span['size']
                if origin in coord_to_index and font in font_to_index:
                    row = coord_to_index[origin]
                    col = font_to_index[font]
                    if matrix[row, col] == 0:
                        matrix[row, col] = []
                    matrix[row, col].append(span)  # Append the entire span dictionary

    # Step 4: Generate the nested dictionary
    nested_dict = {}
    current_title = None
    current_subheader = None

    for row_idx, coord in enumerate(coordinates):
        for col_idx, font in enumerate(fonts):
            if matrix[row_idx, col_idx] != 0:
                spans = matrix[row_idx, col_idx]

                for span in spans:
                    if font == title_font:
                        current_title = span
                        nested_dict[current_title['text']] = {}
                    elif font == subheader_font and current_title:
                        current_subheader = span
                        nested_dict[current_title['text']][current_subheader['text']] = []
                    elif font <= content_max_font and current_subheader:
                        nested_dict[current_title['text']][current_subheader['text']].append(span)

    return nested_dict

def generate_pdf_from_data(data:list, output_path:str):
    """
    Generates a PDF from the nested dictionary data structure.

    Parameters:
        data (dict): The nested dictionary containing sections and fitz spans.
        output_path (str): The file path where the PDF will be saved.
    """
    # Create a new PDF document
    pdf_document = fitz.open()
    
    for section, spans in data.items():
        # Add a new page for each section
        page = pdf_document.new_page()
        text_position = 72  # Initial vertical position (used only for section titles)

        # Add section title
        title_font_size = 14
        page.insert_text(
            (72, text_position),
            section,
            fontsize=title_font_size,
            fontname="helv",
            color=(0, 0, 1),
        )

        # Iterate through each span in the section
        for span in spans:
            bbox = span.get("bbox", [0, 0, 0, 0])  # Use bbox for exact placement

            # Error handling for font issues
            try:
                page.insert_text(
                    (bbox[0], bbox[1]),  # Use bbox coordinates for exact placement
                    span["text"],
                    fontsize=span["size"],
                    fontname="helv",
                    color=tuple(int(span["color"] & 0xFFFFFF) for _ in range(3)),  # Convert span color
                )
            except Exception:
                page.insert_text(
                    (bbox[0], bbox[1]),  # Use bbox coordinates for exact placement
                    span["text"],
                    fontsize=span["size"],
                    fontname="helv",
                    color=(1, 0, 0),  # Fallback color for errors
                )

    # Save the created PDF
    pdf_document.save(output_path)
    pdf_document.close()
    print(f"PDF successfully generated and saved to: {output_path}")

In [ ]:
title_font = 24.0  # Example title font size
subheader_font = 9.0  # Example subheader font size
content_max_font = 8.0  # Example maximum content font size

grand_final_data = {}

for page_data in combined_data: #each page data in doc  
    result = create_matrix_structure(page_data, title_font, subheader_font, content_max_font)
    master_key = list(result.keys()) #fund name
    content = result[master_key[0]] #content of each fund
    
    
    generate_pdf_from_data(content, path + r"\output\samcoDryRun.pdf")
    with pdfplumber.open(path +r"\output\samcoDryRun.pdf") as pdf:
        final_data = []
        final_data_generated = {}
        
        for page in pdf.pages:
            # extract text from the page
            text = page.extract_text()
            final_data.append(text)
        
        #store them in a dict for each page
        for data in final_data:
            content = data.split('\n')
            main_key = content[0]
            values = content[1:]
        
            final_data_generated[main_key] = values
        
        #sort the headers in lex order
        sorted_final_generated = {key: final_data_generated[key] for key in sorted(final_data_generated)}
            
    
    grand_final_data[master_key[0]] = sorted_final_generated
    

In [ ]:
#fund names
list(grand_final_data.keys())

REGEX FUNCTIONS

In [15]:
def return_invest_data(key:str,data:list):
    investment_objective = data
    values = " ".join(txt for txt in investment_objective)

    data = {
        key:values
    }

    return data

def return_scheme_data(key:str,data:list):
    scheme_data = data
    main_key = key
    structured_data = {main_key: {}}

    # Patterns
    date_pattern = r"^(.*?date)\s(\d{2}-[A-Za-z]{3}-\d{4})$"
    benchmark_pattern = r"^(Benchmark)\s+(.*)$"
    application_pattern = r"(?:·)?\d+(?:,\d{3})*(?:\.\d+)?/-"

    for data in scheme_data:
        if re.search(date_pattern, data, re.IGNORECASE):
            match = re.match(date_pattern, data, re.IGNORECASE)
            if match:
                key = match.group(1)
                value = match.group(2)
                structured_data[main_key][key] = value
        elif re.search(benchmark_pattern, data, re.IGNORECASE):
            match = re.match(benchmark_pattern, data, re.IGNORECASE)
            if match:
                key = match.group(1)
                value = match.group(2)
                structured_data[main_key][key] = value
        elif re.search(r"\b(min|application)\b", data, re.IGNORECASE):
            matches = re.findall(application_pattern, data, re.IGNORECASE)
            if matches:
                cleaned_matches = [match.replace('·', '') for match in matches]
                structured_data[main_key]["min_appl_amt"] = cleaned_matches
        elif re.search(r"\b(additional.* and in multiples of)\b", data, re.IGNORECASE):
            matches = re.findall(application_pattern, data, re.IGNORECASE)
            if matches:
                cleaned_matches = [match.replace('·', '') for match in matches]
                structured_data[main_key]["additional_amt"] = cleaned_matches

    return structured_data

def return_fund_data(key:str,data:list):
    fund_manager = data
    main_key = key
    strucuted_data = {main_key:[]}
    current_entry = None
    name_pattern = r'^(Ms\.|Mr\.)'
    manage_pattern = r'^\(|\)$'
    date_pattern = r'\b\w+ \d{1,2}, \d{4}\b'
    experience_pattern = r'^Total Experience: (.+)$'

    for data in fund_manager:
        if re.match(name_pattern,data):
            if current_entry:
                strucuted_data[main_key].append(current_entry)
            current_entry = {
                'name': data.split(",")[0].strip().lower(),
                'designation': "".join(data.split(",")[1:]).strip().lower()
            }
            #print(data.split(",")[0],"".join(data.split(",")[1:]))
        elif re.match(manage_pattern,data):
            if "inception" in data.lower():
                current_entry['managing_since'] = 'inception'
            else:
                date = re.search(date_pattern, data)
                current_entry['managing_since'] = date.group() if date != None else None
        elif re.match(experience_pattern,data):
            current_entry['total_experience'] = data.split(":")[1].strip().lower()
            #print(data.split(":")[1])

        
    if current_entry:  # Append the last entry
        strucuted_data[main_key].append(current_entry)
            
    return strucuted_data

def return_nav_data(key:str,data:list):
    main_key = key
    structured_data = {main_key: {}}
    
    growth_pattern = r"([\w\s]+):\s*·([\d.]+)"
    
    for line in data:
        matches = re.findall(growth_pattern, line)
        for key, value in matches:
            structured_data[main_key][key.strip().lower()] = value
        
    return structured_data

def return_quant_data(key:str,data:list):
    qunatitative_data = data
    main_key = key

    strucuted_data = {main_key:{}}
    current_entry = None
    comment = ""

    ratio_pattern = r"\b(ratio|turnover)\b"
    annual_pattern = r'\b(annualised|YTM)\b'
    macaulay_pattern = r"\b(macaulay.*duration)\b"
    residual_pattern = r"\b(residual.*maturity)\b"
    modified_pattern = r"\b(modified.*duration)\b"

    for data in qunatitative_data:
        if re.search(ratio_pattern,data, re.IGNORECASE):
            key = data.split(":")[0].lower().strip()
            value = data.split(":")[1].lower().strip()
        elif re.search(annual_pattern,data, re.IGNORECASE):
            key = data.split(":")[0].lower().strip()
            value = data.split(":")[1].lower().strip()
        elif re.search(macaulay_pattern,data, re.IGNORECASE):
            key = data.split(":")[0].lower().strip()
            value = data.split(":")[1].lower().strip()
        elif re.search(residual_pattern,data, re.IGNORECASE):
            key = data.split(":")[0].lower().strip()
            value = data.split(":")[1].lower().strip()
        elif re.search(modified_pattern,data, re.IGNORECASE):
            key = data.split(":")[0].lower().strip()
            value = data.split(":")[1].lower().strip()
        else:
            comment+= data
        strucuted_data[main_key][key] = value
    
    strucuted_data[main_key]['comment'] = comment

    return strucuted_data

def return_aum_data(key:str,data:list):
    
    aum = data
    main_key = key
    strucuted_data = {main_key:{}}

    pattern = r"\b\d{1,3}(?:,\d{3})*(?:\.\d+)? Crs\b"

    for data in aum:
        if re.search(r'average', data, re.IGNORECASE):
            match = re.search(pattern, data)
            key = 'avg_aum'
        else:
            match = re.search(pattern, data)
            key = "aum"
        
        strucuted_data[main_key][key] = match.group()

    return strucuted_data

def return_mar_data(key:str,data:list):
    return {
        key: None
    }


In [ ]:
sample = grand_final_data['Samco Active Momentum Fund']
sample

In [14]:
print(sample['Assets Under Management (AUM)'])
print(sample['Fund Manager'])
print(sample['Investment Objective'])
print(sample['NAV as on 31 October 2024 (· per unit)'])
print(sample['Quantitative Data'])
print(sample['Scheme Details'])

['AUM as on October 31, 2024 · 850.06 Crs', 'Average AUM for Month of October 2024 · 852.46 Crs']
['Mr. Paras Matalia, Fund Manager & Head - Research Equity', '(Managing this scheme since inception)', 'Total Experience: Around 9 years', 'Mr. Umeshkumar Mehta, Director, CIO & Fund Manager', '(Managing the scheme since August 01, 2023)', 'Total Experience: Over 20 years', 'Mr. Dhawal Ghanshyam Dhanani', '(Dedicated Fund Manager for Overseas investments since inception)', 'Total Experience: Around 6 years']
['The investment objective of the Scheme is to seek to', 'generate long-term capital appreciation by investing in', 'stocks showing strong momentum. Momentum stocks are', 'such that exhibit positive price momentum · based on the', 'phenomenon that stocks which have performed well in the', 'past relative to other stocks (winners) continue to perform', 'well in the future, and stocks that have performed', 'relatively poorly (losers) continue to perform poorly.', 'However, there can be no

In [16]:
return_aum_data("Assets Under Management (AUM)",sample['Assets Under Management (AUM)'])

{'Assets Under Management (AUM)': {'aum': '850.06 Crs',
  'avg_aum': '852.46 Crs'}}

In [17]:
return_fund_data("Fund Manager",sample['Fund Manager'])

{'Fund Manager': [{'name': 'mr. paras matalia',
   'designation': 'fund manager & head - research equity',
   'managing_since': 'inception',
   'total_experience': 'around 9 years'},
  {'name': 'mr. umeshkumar mehta',
   'designation': 'director cio & fund manager',
   'managing_since': 'August 01, 2023',
   'total_experience': 'over 20 years'},
  {'name': 'mr. dhawal ghanshyam dhanani',
   'designation': '',
   'managing_since': 'inception',
   'total_experience': 'around 6 years'}]}

In [18]:
imp_indices = set()
for k, value in grand_final_data.items():
    for indices in value.keys():
        imp_indices.add(indices)
        
imp_indices = sorted(imp_indices)

function_indices = [return_aum_data,
        return_fund_data,
        return_invest_data,
        return_mar_data,
        return_nav_data,
        return_quant_data,
        return_scheme_data]

function_map = {}

for k, v in zip(imp_indices,function_indices):
    function_map[k] = v

#print(function_map)

def perform_operation(operation, data):
    
    if operation in function_map.keys():
        # Call the mapped function
        return function_map[operation](data)
    else:
        return "Invalid operation"

In [29]:
import pprint, json

grand_dictionary = {}
for master_k in grand_final_data.keys():

    sample = grand_final_data[master_k]
    
    page_content = list()
    
    for main_k, main_content in sample.items():
        
        hello = function_map[main_k]
        page_content.append(hello(main_k, main_content))
    
    grand_dictionary[master_k] = page_content

In [33]:
with open(path +r"\output\dump.json", "w", encoding="utf-8") as file:
    json.dump(grand_dictionary, file, ensure_ascii=False, indent=4)